In [ ]:
#import necessary modules
import os
import numpy as np
import astropy.units as u
import astropy.constants as const
import matplotlib.pyplot as plt
import matplotlib
from multiprocessing import Pool

from IPython.display import Latex
%matplotlib inline

# Orbit Integrator Class

Class created to integrate the orbit of a system with various gravitational potentials and using various integration methods

In [ ]:
class OrbitIntegrator:
    '''calculate an orbit of a system using various leap frog methods'''
    
    def __init__(self, filename): # 
        #Input: filename = name of file where you will store integrated orbit
        
        ### get the gravitational constant (the value is 4.498502151575286e-06)
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        #define the path to store your orbits and snapshots
        self.path = 'insert/path/here'
        
        #string to store which potential you use, uncomment the one you wish to use
        self.potential = 'GalaxyPotential'
        #self.potential = 'KeplerPotential'
        
        #mass of body with potential
        self.M = 9e10  #in solar massess
        
        ### **** store the output file name
        self.filename = filename
        
        #compute initial relative position and velocity of body with potential and
        #its companion
        
        #if circular orbit, can use this and apovec for the vector form
        self.r0mag = 8.0 
        
        #if elliptical orbit, can use these
        #start at apocenter
        self.apovec = np.array([8., 0, 0])
        self.apomag = 8.0
        
        #set variable for pericenter distance 
        self.perimag = 1.0
        
        #set variables for when using a different potential
        
        #for NFW halo potential
        self.Mhalo = 8e11 #in M sun
        self.rshalo = 16.0 #in kpc
        self.c = 15.3
        
        #for Hernquist profile
        self.Mbulge = 5e9 #in Msun
        self.rsbulge = 0.5 #kpc
        
        #for Miyamoto-Nagai disk
        self.Mdisk = 6.8e10 #in Msun
        self.a = 3.0 #kpc
        self.b = 0.28 #kpc 
        
        #varaible to use for initial velocity for circular orbit
        #self.v0 = np.array([0, np.sqrt((self.G*self.M)/self.r0mag), 0])
        
        #variable to use for initial velocity for elliptical orbit
        #self.v0 = np.array([0, np.sqrt(((2*self.G*self.M)*((1/self.perimag)-(1/self.apomag)))/((self.apomag**2/self.perimag**2) - 1.0)), 0])
        
    #calculate the acceleration vector due to the Kepler potential
    def KeplerAccel(self, r):
        '''Inputs:r = radial position vector [x, y, z]
           Outputs: Acceleration vector of kepler potential'''
        r_mag = np.sqrt(r[0]**2 + r[1]**2 + r[2]**2)
        return (-(self.G*self.M)/r_mag**3)*r
    
    #calculate the acceleration vector due to an NFW potential (halo)
    def NFWHaloAccel(self, M, rs, c, r):
        '''Inputs: M = halo mass (in Msun)
                   rs = scale length
                   c = numerical factor
                   r = radial position vector (in kpc)
           Outputs: acceleration vector due to NFW potential'''
        #calculate magnitude of position vector
        rmag = np.sqrt(r[0]**2 + r[1]**2 + r[2]**2)
        
        #store number for all the terms with c into one variable
        A = np.log(1+c) - (c/(1+c))
        
        #store other term parts as other variables to avoid Messing Up
        B = 1/(rs + rmag)
        
        D = np.log(1 + rmag/rs)/rmag
        
        #calculate acceleration
        NFW = r*(self.G*M/(A*rmag**2))*(B-D)
        
        return NFW
        
    #calculate the acceleration vector due to the Hernquist potential (bulge)
    def HernquistAccel(self, M, r_a, pos): # it is easiest if you take as an input the position VECTOR 
        """ function that computes the gravitational acceleration vector induced by a Hernquist profile"""
        #Inputs:
            #M = total component mass
            #r_a = scale length
            #pos = position vector
        #Returns:
            #returns the acceleration vector for a given galaxy component
        ### **** Store the magnitude of the position vector
        rmag = np.linalg.norm(pos)
        
        ### *** Store the Acceleration
        Hern =  -pos*(self.G*M)/(rmag*(r_a + rmag)**2) 
        
        return Hern
    
    #acceleration vector due to the disk
    def MiyamotoNagaiAccel(self, M, a, b, r):# it is easiest if you take as an input a position VECTOR  r 
        """ function that computes the gravitational acceleration vector for the disk using a 
            Miyamoto-Nagai 1975 profile"""
        #Inputs:
            #M = Mass of the disk
            #a = scale disk length
            #b = scale z height
            #r = position vector
        #Returns:
            #returns the acceleration vector for a disk
        
        #calculate smaller parts
        R = np.sqrt(r[0]**2 + r[1]**2)
        B = a + np.sqrt(r[2]**2 + b**2)

        #store the acceleration due to the galactic disk
        
        MNAccel = -(self.G*M/(R**2 + B**2)**1.5) * np.multiply(r,np.array([1,1,B/np.sqrt(B-a)]))
        
       
        return MNAccel 
    
    #total acceleration due to a galaxy's gravitational potential
    def GalaxyAccel(self, r): # input should include the position vector, r
        """function that sums all component acceleration vectors and returns the total 3D acceleration vector"""
        #Input:
            #r = 3D position vector
        #Returns:
            #total acceleration vector for all components of galaxy 
        
        galbulgeaccel = self.HernquistAccel(self.Mbulge, self.rsbulge, r)
        galhaloaccel = self.NFWHaloAccel(self.Mhalo, self.rshalo, self.c, r)
        galdiskaccel = self.MiyamotoNagaiAccel(self.Mdisk, self.a, self.b, r)
        ### Call the previous functions for the halo, bulge and disk
        
        galbulgeandhaloaccel = np.add(galbulgeaccel, galhaloaccel)
        galtotaccel = np.add(galbulgeandhaloaccel, galdiskaccel)
            
            # return the SUM of the output of the acceleration functions - this will return a VECTOR 
        return galtotaccel
        
    
    #calculate the timestep to have it vary along the orbit
    def timestep(self, eta, r, a):
        '''Inputs: eta = numerical factor that determines change in timestep
                   r = radial position vector
                   a = acceleration vector
           Outputs: timestep in Gyr'''
        #compute magnitudes of position and acceleration vectors
        rmag = np.sqrt(r[0]**2 + r[1]**2 + r[2]**2)
        amag = np.sqrt(a[0]**2 + a[1]**2 + a[2]**2)
        
        #compute timestep
        dt = eta*np.sqrt(rmag/amag)
        
        return dt
    
    ##### In the following integration functions, change acceleration function to the gravitational potential desired (KeplerAccel or GalaxyAccel) #####
    
    #leapfrog integrator using kick-drift-kick method
    def KDK(self, dt, r, v):
        """ perform one step kick-drift-kick integration of orbit
        Input: dt time step in Gyr
            r the current position vector [ x, y, z] 
            v the current velocity vector [ vx, vy, vz]
        Returns:  Advances the position and velocity vectors by one timestep """

        # predict the velocity at the next half timestep
        vhalf = v + self.GalaxyAccel(r)*dt/2.0

        # compute the position at the next timestep
        rnext = r + vhalf * dt

        # compute the velocity at the next timestep
        vnext = vhalf + self.GalaxyAccel(rnext)*dt/2.0 

        return rnext, vnext 
        
    #leapfrog integrator using drift-kick-drift method
    def DKD(self, dt, r, v):
        """ perform one step drift-kick-drift integration 
        Input: dt time step in Gyr
            r the current position vector [ x, y, z] 
            v the current velocity vector [ vx, vy, vz]
        Returns:  Advances the position and velocity vectors by one timestep """
        
        # predict the position at the next half timestep
        rhalf = r + v * dt / 2.0
        
        # compute the velocity at the next timestep
        vnext = v + self.GalaxyAccel(rhalf) * dt
        
        # compute the position at the next timestep
        rnext = rhalf + vnext * dt/2.0
        
        return rnext, vnext
    
    #now that we'll be dealing with multiple objects, lets make a function that creates and stores the data for 
    #a cloud of particles 
    def ParticleCloud(self, time, N, mupos, muvel, stdpos, stdvel):
        '''Inputs: time = time of snapshot in Gyr
                   N = number of particles
                   mupos = vector of x, y, z positions you want gaussians to be centered around
                   muvel = vector of vx, vy, vz components you want gaussians to be centered around
                   stdpos = width of gaussian for position components
                   stdvel = width of gaussian for velocity components
           Outputs: a text file of the positions and velocities of every particle in the cloud at the first snapshot in time'''
       
        #create gaussians of each component of position and velocities
        xpos = np.random.normal(mupos[0], stdpos, N)
        ypos = np.random.normal(mupos[1], stdpos, N)
        zpos = np.random.normal(mupos[2], stdpos, N)

        vx = np.random.normal(muvel[0], stdv, N)
        vy = np.random.normal(muvel[1], stdv, N)
        vz = np.random.normal(muvel[2], stdv, N)

        #assign each particle a number 
        particles = np.arange(0, N, 1)
        
        #change it so that the first particle is characteristic of the progenitor
        xpos[0] = mupos[0]
        ypos[0] = 0.0
        zpos[0] = 0.0
        
        vx[0] = 0.0
        vy[0] = muvel[1]
        vz[0] = 0.0
        
        #create an array of all of the data for each particle by using np.column_stack 
        snap_0 = np.column_stack((particles, xpos, ypos, zpos, vx, vy, vz))
        headers = "{:>10s}{:>11s}{:>11s}{:>12s}{:>12s}{:>13s}{:>13s}".format("num", "x", "y", "z", "vx", "vy", "vz")

        # save the data
        with open(self.path + self.potential + "/snaps/snap_0.txt", "w") as f:
            f.write(f'# Time in Gyr = {time:.3f}\n')
            f.write('# ' + headers + '\n')
            np.savetxt(f, snap_0, fmt="%11.3f")

        return snap_0
        

    
    #make the function to integrate the orbit
    def OrbitInt(self, t0, tmax, x, y, z, vx, vy, vz, dt = 0.0, eta = 0.0):
        '''Inputs: t0 = start time of integration
                   tmax = end time of integration
                   x = initial x position in kpc
                   y = initial y position in kpc
                   z = initial z position in kpc
                   vx = initial vx position (in kpc/Gyr)
                   vy = initial vy positon (in kpc/Gyr)
                   vz = initial vz position (in kpc/Gyr)
                   dt = time step forward
                   eta = numerical factor in determining variable timestep 
           Outputs: file of orbit with time, position (x, y, z) and velocity'''

        # initialize the time to the input starting time
        t = t0
        
       # if dt is constant: initialize an empty array of size :  rows int(tmax/dt)+2  , columns 7; can be altered later if it's too big
        if dt != 0.0:
            orbit = np.zeros([int(tmax/dt)+2, 7])
        #otherwise create an orbit file using a minimum dt, then can alter the orbit file later to get rid of extra rows
        else:
            orbit = np.zeros([int(tmax/1e-5),7])
     
        #print(orbit.shape)
        
        # initialize the first row of the orbit
        orbit[0] = t0, x, y, z, vx, vy, vz
        
        # initialize a counter for the orbit.  
        i = 1 # since we already set the 0th values, we start the counter at 1
        
        # start the integration (advancing in time steps and computing LeapFrog at each step)
        while (t < tmax):  # as long as t has not exceeded the maximal time 
            
            #calculate next timestep
            
            #if not given a constant timestep
            if (eta != 0.0):
                dt = self.timestep(eta, orbit[i-1][1:4], self.GalaxyAccel(orbit[i-1][1:4]))
            
            #so that maximum time is not exceeded
            elif (t + dt > tmax):
                dt = tmax - t
                
            # **** advance the time by one timestep, dt
            t = t + dt
            # **** store the new time in the first element of the ith row
            orbit[i,0] = t
            #print(orbit[i,0])
            
            # ***** advance the position and velocity using one of the integration schemes below
            #uncomment the line of code that corresponds to the method you want to use
            # remember that each method returns a position vector and a velocity vector 
            
                
            #using the kick-drift-kick method
            rnew, vnew = self.KDK(dt, orbit[i-1][1:4], orbit[i-1][4:7])
            
            #using the drift-kick-drift method
            #rnew, vnew = self.DKD(dt, orbit[i-1,1:4], orbit[i-1,4:7])
                
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            # and the new velocity vector into the columns with indexes 4, 5, 6 of the ith row of orbit

            orbit[i, 1:4] = rnew
            orbit[i, 4:7] = vnew
            
                        
            # **** update counter i , where i is keeping track of the number of rows (i.e. the number of time steps)
            i = i + 1
        
        #turn orbit into a numpy array
        orbit = np.array(orbit)
        
        #just in case file size is too big, mask out the rows that are all zeros
        condition = (orbit[:,0] != 0.0) & (orbit[:,1] != 0.0) & (orbit[:,2] != 0)
        
        orbit = orbit[condition]
        
        # uncomment to write the data to a file
        #np.savetxt(self.filename, orbit, header = "t x y z vx vy vz", comments='#', fmt = "%.3f")
        return orbit
    
    #create modified version of ParticleCloud and OrbitInt function to advance orbit of multiple particles by one timestep 
    #and create a file of the snapshot in time
    def ParticleCloudStep(self, eta, prevtime, nexttime, snap, snap_num):
        '''Inputs: eta = numerical factor in determining variable timestep 
                   prevtime = previous start time
                   nexttime = time after a timestep has occured
                   snap = string of file of previous snapshot with data
                   snap_num = number of previous snapshot
           Outputs: file out snapshot for the next step in time'''

         # initialize the time to be written in the next snap to be input next time  
        t = nexttime
        
        #read in given snap file, and create arrays of each parameter
        oldsnap = np.genfromtxt(snap, dtype = None, names = True, skip_header = 1)
        partnum = oldsnap['num']
        oldx = oldsnap['x']
        oldy = oldsnap['y']
        oldz = oldsnap['z']
        oldvx = oldsnap['vx']
        oldvy = oldsnap['vy']
        oldvz = oldsnap['vz']
        
        
        # initialize an empty array the same size as the previous snap file
        newsnap = np.zeros([len(oldx), 7])
        
        # since the particle numbers will be the same, initialize the first column to be the same particle numbers 
        #as before
        newsnap[:,0] = partnum
        
        #initialize counter
        i = 0
        
        #lets parallelize to get this to run a bit faster
        with Pool(processes=4) as pool:
            #looping over all of the particles in a given snap file
            while i < len(partnum):
                #calculate the orbit of the particle from the beginning of the time bin to the end of the time bin
                particleorbit = self.OrbitInt(prevtime, nexttime, oldx[i], oldy[i], oldz[i],\
                                                oldvx[i], oldvy[i], oldvz[i], eta=eta)
                #store the last row (most recent position and velocity information) in the corresponding row of
                #the new snap file
                newsnap[i,1:7] = particleorbit[-1,1:7]

                i = i + 1
        
        #increment snap number
        newsnap_num = snap_num + 1
        
        #create and open textfile
        filename = 'snap_' + str(newsnap_num) + '.txt'
        
        #create format of headers
        headers = "{:>10s}{:>11s}{:>11s}{:>12s}{:>12s}{:>13s}{:>13s}".format("num", "x", "y", "z", "vx", "vy", "vz")
        
        # save the data
        with open(self.path + self.potential + "/snaps/" + str(filename), "w") as f:
            f.write(f'# Time in Gyr = {t:.3f}\n')
            f.write('# ' + headers + '\n')
            np.savetxt(f, newsnap, fmt="%11.3f")
        
        return newsnap
    
    #integrates the orbit of a cloud of particles
    def ParticleCloudInt(self, eta, t0, tmax, snaptimestep):
        '''Inputs: eta = variable timestep number
                t0 = start time of integration
                tmax = end time of integration
                snaptimestep = increments of time to save positions and velocities of particles moving in snapshots
           Outputs: snap shots of particles orbiting in the form of text files'''
        #sets lower and upper time limit for integration
        starttime = t0
        endtime = tmax

        #sets time bin
        prevtime = starttime
        nexttime = prevtime + snaptimestep
        
        #start counter for naming snapshots
        i = 0
        
        #integrate orbits and store in snapshots set by snaptimestep 
        while nexttime <= endtime:
            
            #name snap file
            snapname = 'snap_' + str(i) + '.txt'
            snap = self.path + self.potential + '/snaps/' + snapname
            
            
            #make the next snapshot text file with ParticleCloudStep function
            nextsnap = self.ParticleCloudStep(eta, prevtime, nexttime, snap, i)
            
            #increment integration time by snaptimestep
            prevtime = nexttime
            nexttime = nexttime + snaptimestep
            
            #increment counter
            i = i + 1
        
        
    
        

      



## Orbits using KDK integration
This is integrating a circular orbit using the KDK method in a Keplarian potential with the following initial conditions:

$M = 9*10^{10} M_\odot\$

$r_0 = 8 kpc$

$v_0 = \sqrt{\frac{GM}{r_{mag}}}$ (in the y-direction)


In [ ]:
#create class object
circleorbit = OrbitIntegrator('/insert/path/here.txt')

In [ ]:
#initialize apocenter distance and circular velocity
apo = np.array([8.0, 0.0, 0.0])
vcirc = np.array([0.0, np.sqrt((cloud.G*cloud.M)/cloud.r0mag), 0.0])

#this particular run has a start time of 0 Gyr, end time of 1 Gyr, and timestep of 0.0002Gyr
orbitrun = circleorbit.OrbitInt(0, 1., apo[0],apo[1],apo[2],vcirc[0],vcirc[1],vcirc[2],dt=0.001)

In [ ]:
#plot the orbit in spatial (x-y) coordinates
figure, axes = plt.subplots(1)
axes.plot(orbitrun[:,1],orbitrun[:,2], color = 'blue')
axes.set_aspect(1)
axes.set_xlabel('x position (kpc)')
axes.set_ylabel('y position (kpc)')
axes.set_xticks(np.arange(-8.0, 10.0, 2.0))
#axes.set_yticks(np.arange(-8.0, 10.0, 2.0))
axes.set_title('Circular Orbit in a Keplarian Potential')
plt.show()

### Launch a cloud of massless particles with similar positions and velocities drawn from a Gaussian distribution centered on the original orbit

In [ ]:
#create set of data for positions and velocities drawn from gaussians centered on the original orbit 
#using np.random.normal function

#initialize variables and create class object
cloud = OrbitIntegrator('/insert/path/here.txt')

#initialize apocenter distance and circular velocity as vectors
apo = np.array([8.0, 0.0, 0.0])
vcirc = np.array([0.0, np.sqrt((cloud.G*cloud.M)/cloud.r0mag), 0.0])

#number of particles
N = 100

#standard deviation (width of gaussian) for positions
stdpos = 0.5

#standard deviation (width of gaussian) for velocities
stdv = 0.1


#call function that creates a cloud of particles with the specifications listed above
snap_0 = cloud.ParticleCloud(0.0, N, apo, vcirc, stdpos, stdv)

In [ ]:
#now integrate the orbits for each particle with a varying timestep (set by eta) for the specified amount of time (0-1 gigayear)
cloud.ParticleCloudInt(0.01, 0.0, 1.0, 0.01)

In [ ]:
#now let's plot the snapshots in time!

#function for reading in .txt files
def Read(filename):
    file = open(filename, 'r')     #open/read in file
    line1 = file.readline()       #read in first line of file
    label, value = line1.split('= ')  #split parts of first line into label/value
    time = float(value)  #value of time in Gyr
    return time


import os
from pathlib import Path

#set path to store snapshots
directory_in_str = '/insert/path/to/store/snaps'

pathlist = Path(directory_in_str).glob('**/*.txt')

#iterate through each snap text file and create a plot the positions of the star particles in each
for path in pathlist:
    path_in_str = str(path)
    time = Read(path_in_str)
    filepath, snapnumext = path_in_str.split('_')
    snapnum, extension = snapnumext.split('.txt')
    
    snapdata = np.genfromtxt(path_in_str, dtype = None, names = True, skip_header = 1)
    
    #first read in the data
    snapx = snapdata['x']
    snapy = snapdata['y']
    snapz = snapdata['z']
    snapvx = snapdata['vx']
    snapvy = snapdata['vy']
    snapvz = snapdata['vz']
    
    #now plot it
    fig = plt.figure()
    fig.set_facecolor('black')
    ax1 = fig.add_subplot(1, 1, 1)
    ax1.set_facecolor('black')
    ax1.tick_params(axis='x', colors='white')
    ax1.tick_params(axis='y', colors='white')
    ax1.xaxis.label.set_color('white')
    ax1.yaxis.label.set_color('white')
    ax1.title.set_color('white')
    ax1.set_aspect('1')

    # Set the spine color to white
    for spine in ax1.spines.values():
        spine.set_edgecolor('white')
        
    #then plot all star particles
    ax1.scatter(snapx, snapy, color='w', s = 8, alpha=0.7)
    
    #set axis labels, title, and limits
    ax1.set_xlabel('x (kpc)')
    ax1.set_ylabel('y (kpc)')
    ax1.set_title('Time = %.2f Gyr' % time)

    ax1.set_xlim(-12.0, 12.0)
    ax1.set_ylim(-12.0, 12.0)
    
    #save and plot the snapshots
    fig.savefig(directory_in_str + 'snap_plots/' + 'snap_' + str(snapnum) + '_xyplot.png',dpi = 300)
    plt.show()
    plt.close()
    